In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})

%matplotlib inline

In [5]:
import psycopg2
import pandas as pd

dbparams = {
    'dbname': 'bde',
    'port': 5432,
    'host': 'yuma.hpc.nrel.gov',
    'user': 'redoxops',
    'password': 'R3d0x!',
    'options': f'-c search_path=redox',
}

with psycopg2.connect(**dbparams) as conn:
    redf = pd.read_sql_query("""
    SELECT * from redox_rl_1 where run=4 and status='finished';
    """, conn)
    

In [20]:
import re
import gzip
from tqdm import tqdm
tqdm.pandas()


search = re.compile('<S\*\*2>=\ \d\.\d{4}')

def get_s2(logfile):
    opt_freq = False
    s2 = None
    open_func = gzip.open if logfile.endswith('.gz') else open
    
    with open_func(logfile, 'rt') as f:
        for line in f:
            if 'opt freq' in line:
                opt_freq = True
            if opt_freq:
                for result in search.finditer(line):
                    s2 = float(result.group()[-6:])
                    
    return s2

redf['s2'] = redf.logfile.progress_apply(get_s2)

100%|██████████| 10663/10663 [11:04<00:00, 16.06it/s]


In [22]:
import rdkit
from rdkit import Chem

def get_ac(mol, covalent_factor=1.3):
    """
    Generate adjacent matrix from atoms and coordinates.
    AC is a (num_atoms, num_atoms) matrix with 1 being covalent bond and 0 is not
    covalent_factor - 1.3 is an arbitrary factor
    args:
        mol - rdkit molobj with 3D conformer
    optional
        covalent_factor - increase covalent bond length threshold with facto
    """

    # Calculate distance matrix
    dMat = Chem.Get3DDistanceMatrix(mol)

    pt = Chem.GetPeriodicTable()
    num_atoms = mol.GetNumAtoms()
    Rcovs = [pt.GetRcovalent(a_i.GetAtomicNum())
             for a_i in mol.GetAtoms()]

    for i in range(num_atoms):
        a_i = mol.GetAtomWithIdx(i)
        for j in range(i + 1, num_atoms):
            a_j = mol.GetAtomWithIdx(j)
            if dMat[i, j] <= ((Rcovs[i] + Rcovs[j]) * covalent_factor):
                yield tuple(sorted((i, j)))
                
                
def get_bond_ac(mol):
    return {tuple(sorted((bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()))) for bond in mol.GetBonds()}


def check_different_bonds(row, covalent_factor=1.3):
    mol = rdkit.Chem.MolFromMolBlock(row.mol)
    return len(get_bond_ac(mol).symmetric_difference(set(get_ac(mol, covalent_factor))))

changed_bonds = redf.progress_apply(check_different_bonds, axis=1)

100%|██████████| 10663/10663 [00:12<00:00, 869.23it/s] 


In [34]:
redf[changed_bonds > 0].type.value_counts()

oxidized    329
reduced       7
Name: type, dtype: int64

In [38]:
valid_redf = redf[changed_bonds == 0]
valid_redf = valid_redf.drop(valid_redf[(valid_redf.type == 'radical') & (valid_redf.s2 > 0.8)].index)
valid_redf = valid_redf.drop(valid_redf[(valid_redf.type == 'oxidized') & (valid_redf.s2 > 0.25)].index)
valid_redf = valid_redf.drop(valid_redf[(valid_redf.type == 'reduced') & (valid_redf.s2 > 0.25)].index)

In [40]:
import rdkit
from rdkit.Chem.EnumerateStereoisomers import EnumerateStereoisomers, StereoEnumerationOptions

def get_smiles(molblock):
    mol = rdkit.Chem.MolFromMolBlock(molblock)
    rdkit.Chem.rdmolops.AssignStereochemistryFrom3D(mol)
    return rdkit.Chem.MolToSmiles(mol)
    
valid_redf['stereo_smiles'] = valid_redf.mol.str.replace('2  3\n', '2  0\n').apply(get_smiles)

In [41]:
def num_isomers(smiles):
    opts = StereoEnumerationOptions(unique=True)
    return len(list(EnumerateStereoisomers(rdkit.Chem.MolFromSmiles(smiles), options=opts)))

unique_smiles = pd.DataFrame(valid_redf.stereo_smiles.unique(), columns=['smiles'])
unique_smiles['num_isomers'] = unique_smiles.smiles.apply(num_isomers)

In [42]:
unique_smiles[unique_smiles.num_isomers > 1]

,smiles,num_isomers


In [43]:
radical = valid_redf[valid_redf.type == 'radical'].set_index('stereo_smiles')
oxidized = valid_redf[valid_redf.type == 'oxidized'].set_index('stereo_smiles')
reduced = valid_redf[valid_redf.type == 'reduced'].set_index('stereo_smiles')

e_electronaffinity = -((reduced.freeenergy - radical.freeenergy) * 627.509) / 23.061 - 4.281
e_ionizationenery = -((radical.freeenergy - oxidized.freeenergy) * 627.509) / 23.061 - 4.281

In [44]:
e_ionizationenery.name = 'ionization energy'
e_electronaffinity.name = 'electron affinity'

e_ionizationenery = e_ionizationenery.dropna()
e_electronaffinity = e_electronaffinity.dropna()

df = pd.DataFrame(e_ionizationenery).merge(pd.DataFrame(e_electronaffinity), left_index=True, right_index=True, how='outer')
df.shape

(3698, 2)

In [45]:
df = df.merge(valid_redf.drop_duplicates(subset='stereo_smiles')[['smiles', 'stereo_smiles']], left_index=True, right_on='stereo_smiles')

In [46]:
df.smiles.duplicated().any()

False

In [47]:
df.stereo_smiles.duplicated().any()

False

In [48]:
# test = pd.read_csv('/projects/rlmolecule/pstjohn/spin_gnn/20210109_dft_ml_redox_data.csv.gz')

# x = df.merge(test, left_on='stereo_smiles', right_on='smiles')
# x.head()

In [49]:
fdf = df.rename(columns={'smiles': 'err_smiles', 'stereo_smiles': 'smiles'})

In [50]:
fdf

,ionization energy,electron affinity,err_smiles,smiles
457,0.913684,-0.292055,C#CC1=C(C(C)C)C(C2CC2)=C([O])CC1,C#CC1=C(C(C)C)C(C2CC2)=C([O])CC1
462,1.066037,-0.326042,C#C[CH]C(C)=C(C(=O)O)C(C)(C)C,C#C[CH]/C(C)=C(\C(=O)O)C(C)(C)C
5941,NaN,-0.382259,CC(=C(C=C([O])O)C(C)(C)C)C(C)(C)C,C/C(=C(/C=C(\[O])O)C(C)(C)C)C(C)(C)C
5921,0.844432,-0.547755,CC(=C(C(O)=C[O])C(C)(C)C)C(C)(C)C,C/C(=C(C(/O)=C/[O])\C(C)(C)C)C(C)(C)C
5907,0.843045,-0.576871,CC(=C(C(O)=C[O])C(C)(C)C)C(C)C,C/C(=C(C(/O)=C/[O])\C(C)(C)C)C(C)C
...,...,...,...,...
469,0.242474,-0.291457,[O]C=C(CCO)C1=C(C2CC2)CCCC1,[O]/C=C(\CCO)C1=C(C2CC2)CCCC1
465,0.543834,-0.251810,[O]C=C1CCC=C(CO)C2=C1CCCC2,[O]/C=C1\CCC=C(CO)C2=C1CCCC2
466,0.519780,-0.347701,[O]C=C1CCCCC2=C1CCC=C2CO,[O]/C=C1\CCCCC2=C1CCC=C2CO
463,0.928242,-0.174722,[O]C1=C(CCCO)C(CO)=CC=CC1,[O]C1=C(CCCO)C(CO)=CC=CC1


In [51]:
fdf.to_csv('/projects/rlmolecule/pstjohn/spin_gnn/20210216_fixed_rl_redox_data.csv', index=False)

In [46]:
fdf

,ionization energy,electron affinity,err_smiles,smiles
457,0.913684,-0.292055,C#CC1=C(C(C)C)C(C2CC2)=C([O])CC1,C#CC1=C(C(C)C)C(C2CC2)=C([O])CC1
462,1.066037,-0.326042,C#C[CH]C(C)=C(C(=O)O)C(C)(C)C,C#C[CH]/C(C)=C(\C(=O)O)C(C)(C)C
5941,-0.883864,-0.382259,CC(=C(C=C([O])O)C(C)(C)C)C(C)(C)C,C/C(=C(/C=C(\[O])O)C(C)(C)C)C(C)(C)C
5921,0.844432,-0.547755,CC(=C(C(O)=C[O])C(C)(C)C)C(C)(C)C,C/C(=C(C(/O)=C/[O])\C(C)(C)C)C(C)(C)C
5907,0.843045,-0.576871,CC(=C(C(O)=C[O])C(C)(C)C)C(C)C,C/C(=C(C(/O)=C/[O])\C(C)(C)C)C(C)C
...,...,...,...,...
469,0.242474,-0.291457,[O]C=C(CCO)C1=C(C2CC2)CCCC1,[O]/C=C(\CCO)C1=C(C2CC2)CCCC1
465,0.543834,-0.251810,[O]C=C1CCC=C(CO)C2=C1CCCC2,[O]/C=C1\CCC=C(CO)C2=C1CCCC2
466,0.519780,-0.347701,[O]C=C1CCCCC2=C1CCC=C2CO,[O]/C=C1\CCCCC2=C1CCC=C2CO
463,0.928242,-0.174722,[O]C1=C(CCCO)C(CO)=CC=CC1,[O]C1=C(CCCO)C(CO)=CC=CC1


In [258]:
spin_bv = pd.read_csv('/projects/rlmolecule/pstjohn/spin_gnn/20210109_dft_ml_spin_bv_data.csv.gz')

In [263]:
spin_bv.shape

(54863, 6)

In [281]:
spin_bv_fixed = spin_bv.merge(fdf, left_on='smiles', right_on='err_smiles', how='inner').drop('smiles_x', 1).rename(columns={'smiles_y': 'smiles'})
spin_bv_fixed.to_csv('/projects/rlmolecule/pstjohn/spin_gnn/20210211_fixed_rl_spin_bv_data.csv.gz', index=False, compression='gzip')

In [283]:
spin_bv_fixed

,buried_vol,pred buried_vol,fractional_spin,pred fractional_spin,atom_index,ionization energy,electron affinity,err_smiles,smiles
0,40.057887,37.746520,0.085155,0.130998,0,0.194136,NaN,C=C(CO)C1=C([CH]O)C(=O)C=C(C)C1=O,C=C(CO)C1=C([CH]O)C(=O)C=C(C)C1=O
1,54.431655,53.257458,0.024316,0.055328,1,0.194136,NaN,C=C(CO)C1=C([CH]O)C(=O)C=C(C)C1=O,C=C(CO)C1=C([CH]O)C(=O)C=C(C)C1=O
2,41.549793,40.567318,0.009385,0.004386,2,0.194136,NaN,C=C(CO)C1=C([CH]O)C(=O)C=C(C)C1=O,C=C(CO)C1=C([CH]O)C(=O)C=C(C)C1=O
3,32.768445,32.566360,0.001184,0.002064,3,0.194136,NaN,C=C(CO)C1=C([CH]O)C(=O)C=C(C)C1=O,C=C(CO)C1=C([CH]O)C(=O)C=C(C)C1=O
4,62.511166,63.582790,0.345404,0.263097,4,0.194136,NaN,C=C(CO)C1=C([CH]O)C(=O)C=C(C)C1=O,C=C(CO)C1=C([CH]O)C(=O)C=C(C)C1=O
...,...,...,...,...,...,...,...,...,...
54288,41.773301,41.901000,0.003670,0.002244,9,NaN,0.147084,CC(C)(C)C1=C([CH]O)CCC(=O)C1=O,CC(C)(C)C1=C([CH]O)CCC(=O)C1=O
54289,45.459577,45.216320,0.000288,0.004649,10,NaN,0.147084,CC(C)(C)C1=C([CH]O)CCC(=O)C1=O,CC(C)(C)C1=C([CH]O)CCC(=O)C1=O
54290,31.590982,31.469965,0.012289,0.006706,11,NaN,0.147084,CC(C)(C)C1=C([CH]O)CCC(=O)C1=O,CC(C)(C)C1=C([CH]O)CCC(=O)C1=O
54291,56.270252,58.166733,0.013208,0.017550,12,NaN,0.147084,CC(C)(C)C1=C([CH]O)CCC(=O)C1=O,CC(C)(C)C1=C([CH]O)CCC(=O)C1=O
